In [2]:
import pandas as pd
from pandas import read_csv
import numpy as np

In [23]:
# power_df_origin
power_df_origin = pd.read_excel('./data/power_20200428_20210429.xlsx', sheet_name=2, converters={'소규모발전기ID':str})
power_df_origin = power_df_origin.sort_values('번호')
print(power_df_origin.columns)
print(power_df_origin)

Index(['번호', '중개사업자명', '집합전력자원명', '소규모발전기명', '소규모발전기ID', '전력시장발전기명', '전력시장ID',
       '설비용량(kW)', '날짜', '계량시간', '발전량'],
      dtype='object')
        번호  중개사업자명    집합전력자원명     소규모발전기명 소규모발전기ID   전력시장발전기명 전력시장ID  \
4519     1  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
4520     2  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
4521     3  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
4522     4  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
4523     5  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
...    ...     ...        ...         ...      ...        ...    ...   
8748  8749  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
8749  8750  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
8750  8751  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
8751  8752  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE98   
8752  8753  (주)엘시스  elsys0001  쏠3호 태양광발전소    00236  쏠3호태양광발전소   AE

In [24]:
# power_df 
power_df = power_df_origin.copy()
power_df.drop(['번호', '중개사업자명', '집합전력자원명', '소규모발전기명', '전력시장발전기명'], axis=1, inplace=True)

power_df = power_df.rename(columns={"소규모발전기ID": "generatorID"})
power_df = power_df.rename(columns={"전력시장ID": "marketID"})
power_df = power_df.rename(columns={"설비용량(kW)": "capacity"})
power_df = power_df.rename(columns={"날짜": "date"})
power_df = power_df.rename(columns={"계량시간": "hour"})
power_df = power_df.rename(columns={"발전량": "power"})
print(power_df)

     generatorID marketID  capacity       date  hour  power
4519       00236     AE98        90 2021-04-29     0    0.0
4520       00236     AE98        90 2021-04-29     1    0.0
4521       00236     AE98        90 2021-04-29     2    0.0
4522       00236     AE98        90 2021-04-29     3    0.0
4523       00236     AE98        90 2021-04-29     4    0.0
...          ...      ...       ...        ...   ...    ...
8748       00236     AE98        90 2020-04-28    19    0.0
8749       00236     AE98        90 2020-04-28    20    0.0
8750       00236     AE98        90 2020-04-28    21    0.0
8751       00236     AE98        90 2020-04-28    22    0.0
8752       00236     AE98        90 2020-04-28    23    0.0

[8753 rows x 6 columns]


In [38]:
import os
import fnmatch
from pandas import read_csv

def getWeather():
# pow 파일 load
    for filename in os.listdir('./data/'):
        if fnmatch.fnmatch(filename, '*ASOS_*.csv'):
            print(filename)

            # load csv data
            weather_df_origin = read_csv('./data/'+filename, encoding='CP949')
            weather_df = weather_df_origin.copy()
            print(weather_df.columns)
            
            weather_df.drop(['지점','지점명'], axis=1, inplace=True)
            weather_df.drop(['기온 QC플래그','강수량 QC플래그','풍속 QC플래그','풍향 QC플래그','습도 QC플래그'], axis=1, inplace=True)
            weather_df.drop(['현지기압 QC플래그','해면기압 QC플래그','일조 QC플래그','지면온도 QC플래그'], axis=1, inplace=True)
            weather_df.drop(['5cm 지중온도(°C)','10cm 지중온도(°C)','20cm 지중온도(°C)','30cm 지중온도(°C)'], axis=1, inplace=True)
            weather_df.drop(['3시간신적설(cm)','일사(MJ/m2)','운형(운형약어)','지면상태(지면상태코드)','현상번호(국내식)'], axis=1, inplace=True)

            print(weather_df.columns)
            # set column name
            weather_df.columns = ['date','temperature','precipitation','windSpeed','windDirection','humidity','vaporPressure', 'dewPoint'
                                  ,'localPressure','seaSurfacePressure','sunshine','Snow','cloudVolume','middleLowerLayer Cloud volume (10th quartile)','Lowest cloud height (100m)','Visibility (10m)','Ground temperature']

            print(weather_df.columns)
            # prioirty sort (피어슨상관계수)
            dataset = dataset[['ymdhms','sunshine','humdt','wnd_spd','visiblt','cloud2',
                               'cloud','grd_temprt','wnd_dir','dewpnt','steampressr','temprt',
                               'mincloud','rain','pressr','seapressr','snow']]


            # set NA data (관측값 0이 누적되어 결측된 경우. 0으로 세팅)
            dataset['rain'].fillna(0, inplace=True)     #강수량
            dataset['sunshine'].fillna(0, inplace=True) #일조
            dataset['snow'].fillna(0, inplace=True)     #적설량

            #일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
            dataset['ymdhms'] = dataset['ymdhms'].str[0:4]+dataset['ymdhms'].str[5:7]+dataset['ymdhms'].str[8:10]+dataset['ymdhms'].str[-5:-4].replace(" ","0")+dataset['ymdhms'].str[-4:-3]
            # pow측정값 중 결측값 많은 일자 제거
            dataset = dataset[(dataset['ymdhms'].str[0:8]>=date_start) & (dataset['ymdhms'].str[0:8]<date_end)]
            for err_date in err_date_list:
                idx_err = dataset[dataset['ymdhms'].str.startswith(err_date)].index
                dataset = dataset.drop(idx_err)

            #낮시간 추출 (5~20시)
            dataset = dataset[(dataset['ymdhms'].str[-2:]>=str(powhr_start).rjust(2, '0')) &(dataset['ymdhms'].str[-2:]<=str(powhr_end))]
            dataset = dataset.interpolate(method='linear')# 결측값 보간
            
            # save file (test용)
            dataset.to_csv(save_path+"/weather_"+save_name+".csv",mode='w',index=False)

            # normalization
            dataset.drop(['ymdhms'], axis=1, inplace=True)
            dataset = dataset.astype('float32')
            dataset = dataset.interpolate(method='linear')
            
            #YEO-JOHNSON transform
            #yeo_df = yeo_johnson_transform(dataset)
            yeo_df = dataset
            #insert feature (test)
            yeo_df.insert(2, 'temp_press', yeo_df['temprt']-yeo_df['steampressr'], True)
            yeo_df.insert(2, 'sunshine_humdt', abs(yeo_df['sunshine'])-(yeo_df['humdt']*(2.1)), True)#0.35
            
            sc = MinMaxScaler(feature_range = (0, 1))#scale
            scaled_weather = sc.fit_transform(yeo_df.values)
            weather = pd.DataFrame(scaled_weather, columns=yeo_df.columns, index=list(yeo_df.index.values))
            
            weather = weather.iloc[:, 0:data_dim] #feature size 조절
    return weather

print(getWeather())
    

ASOS_20200428_20210429.csv
Index(['지점', '지점명', '일시', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그',
       '풍속(m/s)', '풍속 QC플래그', '풍향(16방위)', '풍향 QC플래그', '습도(%)', '습도 QC플래그',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '현지기압 QC플래그', '해면기압(hPa)',
       '해면기압 QC플래그', '일조(hr)', '일조 QC플래그', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)',
       '전운량(10분위)', '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)',
       '지면상태(지면상태코드)', '현상번호(국내식)', '지면온도(°C)', '지면온도 QC플래그', '5cm 지중온도(°C)',
       '10cm 지중온도(°C)', '20cm 지중온도(°C)', '30cm 지중온도(°C)'],
      dtype='object')
Index(['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '적설(cm)', '전운량(10분위)',
       '중하층운량(10분위)', '최저운고(100m )', '시정(10m)', '지면온도(°C)'],
      dtype='object')
Index(['Date', 'Air temperature (°C)', 'Precipitation (mm)',
       'Wind speed (m/s)', 'Wind direction (16 directions)', 'Humidity (%)',
       'Vapor pressure (hPa)', 'Dew point temperature (°C)',
   

UnboundLocalError: local variable 'dataset' referenced before assignment

In [ ]:
print(power_df.columns)
power_df.columns = ['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
        'Temperature', 'Precipitation', 'Wind speed', 'Wind direction', 'Humidity',
       'Vapor pressure','Dew point','Local pressure','Sea pressure','Sunlight',
        'Irradiance','Snowfall','Total cloud',
        'Visibility','Ground temperature']
print(testDF.columns)

incheon_wether = incheon_wether.rename(columns={"일시": "YMDH"})

In [66]:
wether = read_csv('./data/weather.csv', encoding='CP949')
incheon_wether = read_csv('./data/incheon_weather.csv',encoding='CP949')
#korea_pow = read_csv('./data/korea_pow.csv', encoding='CP949')
korea_pow = read_csv('./data/korea_pow.csv', encoding='CP949')

In [67]:
print(korea_dust)
#print(incheon_dust)
print(wether)

korea_pow["YMDH"] = korea_pow["거래일"] +korea_pow["시간"].astype(str).str.zfill(2)

       STATION_ID STATION_NAME        YMDH    SO2   CO     O3    NO2   PM10  \
0          111121       junggu  2015010101  0.006  0.6  0.022  0.011   44.0   
1          111121       junggu  2015010102  0.006  0.6  0.022  0.010   57.0   
2          111121       junggu  2015010103  0.006  0.6  0.021  0.010   76.0   
3          111121       junggu  2015010104  0.006  0.6  0.022  0.009   86.0   
4          111121       junggu  2015010105  0.006  0.5  0.021  0.009  106.0   
...           ...          ...         ...    ...  ...    ...    ...    ...   
51883      111121       junggu  2020103120  0.003  0.4  0.021  0.035   34.0   
51884      111121       junggu  2020103121  0.003  0.4  0.025  0.028   32.0   
51885      111121       junggu  2020103122  0.003  0.4  0.030  0.023   30.0   
51886      111121       junggu  2020103123  0.003  0.4  0.034  0.019   27.0   
51887      111121       junggu  2020103124  0.003  0.4  0.033  0.019   27.0   

       PM25  
0       7.0  
1      10.0  
2      10

In [68]:
korea_pow['YMDH'] = korea_pow['YMDH'].str.replace('-', '', regex=False)

In [69]:
korea_pow['month'] = korea_pow['YMDH'].str.slice(start=4, stop=6)
korea_pow['fullday'] = korea_pow['YMDH'].str.slice(start=0, stop=-2)
korea_pow['month'] = korea_pow['month'].astype('int')
korea_pow['season'] = "winter"
korea_pow['year'] = korea_pow['YMDH'].str.slice(start=0, stop=4)

korea_pow.loc[(korea_pow['month'] >=3) & (korea_pow['month'] <=5),'season'] = 'spring'
korea_pow.loc[(korea_pow['month'] >=6) & (korea_pow['month'] <=8),'season'] = 'summer'
korea_pow.loc[(korea_pow['month'] >=9) & (korea_pow['month'] <=11),'season'] = 'autumn'

In [70]:

wether['일시'] = wether['일시'].str.replace('-', '', regex=False)
wether['일시'] = wether['일시'].str.replace(':00', '', regex=False)
wether['일시'] = wether['일시'].str.replace(' ', '', regex=False)

incheon_wether['일시'] = incheon_wether['일시'].str.replace('-', '', regex=False)
incheon_wether['일시'] = incheon_wether['일시'].str.replace(':00', '', regex=False)
incheon_wether['일시'] = incheon_wether['일시'].str.replace(' ', '', regex=False)

In [71]:
wether = wether.rename(columns={"일시": "YMDH"})

incheon_wether = incheon_wether.rename(columns={"일시": "YMDH"})

incheon_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})
kangil_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})
kangdong_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})


#부산 서울 제주 인천

In [72]:
wether

,지점,지점명,YMDH,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,108,서울,2014010100,3.3,NaN,NaN,3.8,250.0,65.0,5.0,-2.6,1001.9,1012.5,NaN,NaN,NaN,NaN,6.0,6.0,Sc,10.0,600.0,NaN,40.0,0.0,0.1,-0.2,0.0,1.5
1,108,서울,2014010101,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
2,108,서울,2014010102,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
3,108,서울,2014010103,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
4,108,서울,2014010104,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230775,184,제주,2020073119,27.4,NaN,NaN,1.5,20.0,86.0,31.3,24.8,1011.4,1013.8,0.0,0.29,NaN,NaN,NaN,9.0,NaN,67.0,1975.0,NaN,NaN,29.3,29.3,30.8,29.3,27.7
230776,184,제주,2020073120,26.6,NaN,NaN,0.6,270.0,91.0,31.7,25.0,1011.4,1013.8,0.0,0.03,NaN,NaN,7.0,7.0,NaN,67.0,1653.0,NaN,NaN,28.2,29.1,30.3,29.3,27.8
230777,184,제주,2020073121,26.3,NaN,NaN,1.2,20.0,92.0,31.3,24.8,1011.5,1013.9,NaN,NaN,NaN,NaN,7.0,0.0,Ci,NaN,1791.0,NaN,NaN,27.5,28.9,30.0,29.3,27.8
230778,184,제주,2020073122,26.9,NaN,NaN,1.3,50.0,85.0,30.0,24.1,1011.5,1013.9,NaN,NaN,NaN,NaN,4.0,0.0,NaN,NaN,2000.0,NaN,NaN,26.9,28.7,29.7,29.2,27.9


In [73]:
seoul_pow = korea_pow.loc[korea_pow['지역세부구분']=='서울시', :]
#korea_pow.loc[korea_pow['지역세부구분']=='강원도', :]
pusan_pow = korea_pow.loc[korea_pow['지역세부구분']=='경상남도', :]
jeju_pow = korea_pow.loc[korea_pow['지역세부구분']=='제주도', :]

In [74]:
seoul_weather = wether.loc[wether['지점명']=='서울', :]
pusan_weather = wether.loc[wether['지점명']=='부산', :]
jeju_weather = wether.loc[wether['지점명']=='제주', :]

In [75]:
jeju_weather.shape

(57694, 29)

In [76]:
seoul_dataset = seoul_pow.merge(seoul_weather, how='left', on='YMDH')
pusan_dataset = pusan_pow.merge(pusan_weather, how='left', on='YMDH')
jeju_dataset = jeju_pow.merge(jeju_weather, how='left', on='YMDH')

In [77]:
seoul_dataset

,연료원,지역세부구분,거래일,시간,전력거래량,YMDH,month,fullday,season,year,지점,지점명,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,태양광,서울시,2014-01-01,1,0.000000,2014010101,1,20140101,winter,2014,108,서울,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
1,태양광,서울시,2014-01-01,2,0.000000,2014010102,1,20140101,winter,2014,108,서울,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
2,태양광,서울시,2014-01-01,3,0.000000,2014010103,1,20140101,winter,2014,108,서울,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
3,태양광,서울시,2014-01-01,4,0.000000,2014010104,1,20140101,winter,2014,108,서울,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
4,태양광,서울시,2014-01-01,5,0.000000,2014010105,1,20140101,winter,2014,108,서울,0.6,NaN,NaN,2.0,270.0,56.0,3.6,-7.1,1002.1,1012.8,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,800.0,NaN,42.0,-0.7,0.0,-0.2,0.1,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57691,태양광,서울시,2020-07-31,20,0.061932,2020073120,7,20200731,summer,2020,108,서울,27.0,NaN,9.0,2.8,250.0,83.0,29.5,23.8,1002.8,1012.5,0.0,0.09,NaN,NaN,9.0,8.0,NaN,5.0,1909.0,NaN,NaN,25.7,27.6,26.9,26.2,25.5
57692,태양광,서울시,2020-07-31,21,0.000000,2020073121,7,20200731,summer,2020,108,서울,26.7,NaN,9.0,2.2,270.0,86.0,30.0,24.1,1003.0,1012.7,NaN,NaN,NaN,NaN,7.0,7.0,Sc,6.0,1519.0,NaN,NaN,25.3,27.3,26.8,26.2,25.5
57693,태양광,서울시,2020-07-31,22,0.000000,2020073122,7,20200731,summer,2020,108,서울,26.5,NaN,9.0,1.8,250.0,87.0,30.0,24.1,1003.1,1012.8,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,1381.0,NaN,NaN,25.4,27.1,26.6,26.1,25.6
57694,태양광,서울시,2020-07-31,23,0.000000,2020073123,7,20200731,summer,2020,108,서울,26.2,0.2,NaN,2.4,200.0,90.0,30.6,24.4,1003.1,1012.8,NaN,NaN,NaN,NaN,8.0,8.0,Sc,4.0,852.0,NaN,1901.0,25.3,26.9,26.4,26.1,25.6


In [78]:
seoul_weather

,지점,지점명,YMDH,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,108,서울,2014010100,3.3,NaN,NaN,3.8,250.0,65.0,5.0,-2.6,1001.9,1012.5,NaN,NaN,NaN,NaN,6.0,6.0,Sc,10.0,600.0,NaN,40.0,0.0,0.1,-0.2,0.0,1.5
1,108,서울,2014010101,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
2,108,서울,2014010102,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
3,108,서울,2014010103,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
4,108,서울,2014010104,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215439,108,서울,2020073119,27.4,NaN,9.0,1.8,230.0,82.0,29.8,24.0,1002.6,1012.3,0.0,0.15,NaN,NaN,9.0,9.0,NaN,5.0,1995.0,NaN,NaN,26.6,27.9,27.1,26.2,25.4
215440,108,서울,2020073120,27.0,NaN,9.0,2.8,250.0,83.0,29.5,23.8,1002.8,1012.5,0.0,0.09,NaN,NaN,9.0,8.0,NaN,5.0,1909.0,NaN,NaN,25.7,27.6,26.9,26.2,25.5
215441,108,서울,2020073121,26.7,NaN,9.0,2.2,270.0,86.0,30.0,24.1,1003.0,1012.7,NaN,NaN,NaN,NaN,7.0,7.0,Sc,6.0,1519.0,NaN,NaN,25.3,27.3,26.8,26.2,25.5
215442,108,서울,2020073122,26.5,NaN,9.0,1.8,250.0,87.0,30.0,24.1,1003.1,1012.8,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,1381.0,NaN,NaN,25.4,27.1,26.6,26.1,25.6


In [79]:
korea_dust['YMDH'] = korea_dust['YMDH'].astype(str)
korea_dust['fullday'] = korea_dust['YMDH'].str.slice(start=0, stop=-2)
korea_dust['month'] = korea_dust['YMDH'].str.slice(start=4, stop=6)

korea_dust['month'] = korea_dust['month'].astype('int')
korea_dust['season'] = "winter"
korea_dust['year'] = korea_dust['YMDH'].str.slice(start=0, stop=4)

korea_dust.loc[(korea_dust['month'] >=3) & (korea_dust['month'] <=5),'season'] = 'spring'
korea_dust.loc[(korea_dust['month'] >=6) & (korea_dust['month'] <=8),'season'] = 'summer'
korea_dust.loc[(korea_dust['month'] >=9) & (korea_dust['month'] <=11),'season'] = 'autumn'




#kangdong_dust = read_csv('./data/kangdong_dust.csv', encoding='CP949')

kangdong_dust['YMDH'] = kangdong_dust['YMDH'].astype(str)
kangdong_dust['fullday'] = kangdong_dust['YMDH'].str.slice(start=0, stop=-2)
kangdong_dust['month'] = kangdong_dust['YMDH'].str.slice(start=4, stop=6)

kangdong_dust['month'] = kangdong_dust['month'].astype('int')
kangdong_dust['season'] = "winter"
kangdong_dust['year'] = kangdong_dust['YMDH'].str.slice(start=0, stop=4)

kangdong_dust.loc[(kangdong_dust['month'] >=3) & (kangdong_dust['month'] <=5),'season'] = 'spring'
kangdong_dust.loc[(kangdong_dust['month'] >=6) & (kangdong_dust['month'] <=8),'season'] = 'summer'
kangdong_dust.loc[(kangdong_dust['month'] >=9) & (kangdong_dust['month'] <=11),'season'] = 'autumn'

#incheon_dust = read_csv('./data/incheon_dust.csv', encoding='CP949')

incheon_dust['YMDH'] = incheon_dust['YMDH'].astype(str)
incheon_dust['fullday'] = incheon_dust['YMDH'].str.slice(start=0, stop=-2)
incheon_dust['month'] = incheon_dust['YMDH'].str.slice(start=4, stop=6)

incheon_dust['month'] = incheon_dust['month'].astype('int')
incheon_dust['season'] = "winter"
incheon_dust['year'] = incheon_dust['YMDH'].str.slice(start=0, stop=4)

incheon_dust.loc[(incheon_dust['month'] >=3) & (incheon_dust['month'] <=5),'season'] = 'spring'
incheon_dust.loc[(incheon_dust['month'] >=6) & (incheon_dust['month'] <=8),'season'] = 'summer'
incheon_dust.loc[(incheon_dust['month'] >=9) & (incheon_dust['month'] <=11),'season'] = 'autumn'


#kangil_dust = read_csv('./data/kangil_dust.csv', encoding='CP949')

kangil_dust['YMDH'] = kangil_dust['YMDH'].astype(str)
kangil_dust['fullday'] = kangil_dust['YMDH'].str.slice(start=0, stop=-2)
kangil_dust['month'] = kangil_dust['YMDH'].str.slice(start=4, stop=6)

kangil_dust['month'] = kangil_dust['month'].astype('int')
kangil_dust['season'] = "winter"
kangil_dust['year'] = kangil_dust['YMDH'].str.slice(start=0, stop=4)

kangil_dust.loc[(kangil_dust['month'] >=3) & (kangil_dust['month'] <=5),'season'] = 'spring'
kangil_dust.loc[(kangil_dust['month'] >=6) & (kangil_dust['month'] <=8),'season'] = 'summer'
kangil_dust.loc[(kangil_dust['month'] >=9) & (kangil_dust['month'] <=11),'season'] = 'autumn'



In [80]:
kor_dust_dataset = korea_dust.merge(seoul_weather, how='left', on='YMDH')
kangil_dust_dataset = kangil_dust.merge(seoul_weather, how='left', on='YMDH')
kangdong_dust_dataset = kangdong_dust.merge(seoul_weather, how='left', on='YMDH')

incheon_dust_dataset = incheon_dust.merge(incheon_wether, how='left', on='YMDH')

In [82]:

seoul_dataset = seoul_dataset.fillna(0)
pusan_dataset = pusan_dataset.fillna(0)
jeju_dataset = jeju_dataset.fillna(0)



seoul_dataset.to_csv('./data/seoul_pow2.csv', encoding='CP949')
pusan_dataset.to_csv('./data/pusan_pow2.csv', encoding='CP949')
jeju_dataset.to_csv('./data/jeju_pow2.csv', encoding='CP949')

In [95]:
testDF = jeju_dataset.copy()
testDF.columns = jeju_dataset.columns
print(testDF.columns)

testDF.drop(['연료원', '지역세부구분','강수량 QC플래그','지면상태(지면상태코드)', '현상번호(국내식)', '운형(운형약어)','10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)','3시간신적설(cm)','지점', '지점명','중하층운량(10분위)', '최저운고(100m )','5cm 지중온도(°C)'], axis=1, inplace=True)

print(testDF.columns)
testDF.columns = ['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
        'Temperature', 'Precipitation', 'Wind speed', 'Wind direction', 'Humidity',
       'Vapor pressure','Dew point','Local pressure','Sea pressure','Sunlight',
        'Irradiance','Snowfall','Total cloud',
        'Visibility','Ground temperature']
print(testDF.columns)
testDF.to_csv('./data/jeju_pow3.csv', encoding='CP949')


Index(['연료원', '지역세부구분', '거래일', '시간', '전력거래량', 'YMDH', 'month', 'fullday',
       'season', 'year', '지점', '지점명', '기온(°C)', '강수량(mm)', '강수량 QC플래그',
       '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)',
       '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)',
       '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)',
       '현상번호(국내식)', '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)'],
      dtype='object')
Index(['거래일', '시간', '전력거래량', 'YMDH', 'month', 'fullday', 'season', 'year',
       '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)',
       '전운량(10분위)', '시정(10m)', '지면온도(°C)'],
      dtype='object')
Index(['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
       'Temperature', 'Precipitation', 'Wind speed', 'Wind direction',
       'Humidity', 'Vapor pressure', 'Dew point', '

In [96]:
jeju_dataset = read_csv('./data/jeju_pow3.csv')
jeju_dataset

,Unnamed: 0,date,hour,power,YMDH,month,fullday,season,year,Temperature,Precipitation,Wind speed,Wind direction,Humidity,Vapor pressure,Dew point,Local pressure,Sea pressure,Sunlight,Irradiance,Snowfall,Total cloud,Visibility,Ground temperature
0,0,2014-01-01,1,0.000000,2014010101,1,20140101,winter,2014,11.3,0.0,7.1,290.0,51.0,6.8,1.5,1013.8,1016.3,0.0,0.00,0.0,0.0,0.0,6.5
1,1,2014-01-01,2,0.000000,2014010102,1,20140101,winter,2014,10.7,0.0,7.0,290.0,51.0,6.5,0.9,1014.0,1016.5,0.0,0.00,0.0,0.0,0.0,6.5
2,2,2014-01-01,3,0.000000,2014010103,1,20140101,winter,2014,10.4,0.0,5.7,290.0,47.0,5.9,-0.4,1014.2,1016.7,0.0,0.00,0.0,0.0,1500.0,5.7
3,3,2014-01-01,4,0.000000,2014010104,1,20140101,winter,2014,10.5,0.0,6.4,290.0,47.0,6.0,-0.3,1014.0,1016.5,0.0,0.00,0.0,0.0,1200.0,5.5
4,4,2014-01-01,5,0.000000,2014010105,1,20140101,winter,2014,10.4,0.0,5.0,290.0,54.0,6.8,1.4,1013.5,1016.0,0.0,0.00,0.0,2.0,1200.0,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57691,57691,2020-07-31,20,10.137453,2020073120,7,20200731,summer,2020,26.6,0.0,0.6,270.0,91.0,31.7,25.0,1011.4,1013.8,0.0,0.03,0.0,7.0,1653.0,28.2
57692,57692,2020-07-31,21,7.378050,2020073121,7,20200731,summer,2020,26.3,0.0,1.2,20.0,92.0,31.3,24.8,1011.5,1013.9,0.0,0.00,0.0,7.0,1791.0,27.5
57693,57693,2020-07-31,22,2.683048,2020073122,7,20200731,summer,2020,26.9,0.0,1.3,50.0,85.0,30.0,24.1,1011.5,1013.9,0.0,0.00,0.0,4.0,2000.0,26.9
57694,57694,2020-07-31,23,0.376994,2020073123,7,20200731,summer,2020,27.3,0.0,0.3,0.0,79.0,28.6,23.3,1011.4,1013.8,0.0,0.00,0.0,6.0,2000.0,26.6


In [97]:
#seoul_dataset.to_csv('seoul_pow.csv')
#pusan_dataset.to_csv('pusan_pow.csv')
#jeju_dataset.to_csv('jeju_pow.csv')
#kor_dust_dataset.to_csv('kor_dust.csv')
pd.set_option('display.max_columns', 500)


In [98]:
def f(x):
    tmpdict = dict()
    columns = x.columns
    for CL in columns:
        #print(x[CL].dtype)
        if x[CL].dtype=='object':
            x[CL] = x[CL].apply(str)
            tmpdict[CL] = x[CL].max()
        else:
            tmpdict[CL] = x[CL].max()
    #print(tmpdict)
    
    return pd.Series(tmpdict)

seoul_dataset_day_df = seoul_dataset.groupby(by=["fullday"], dropna=False).apply(f)
pusan_dataset_day_df = pusan_dataset.groupby(by=["fullday"], dropna=False).apply(f)
jeju_dataset_day_df = jeju_dataset.groupby(by=["fullday"], dropna=False).apply(f)
kor_dust_dataset_day_df = kor_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)

kangil_dust_dataset_day_df = kangil_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)
kangdong_dust_dataset_day_df = kangdong_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)
incheon_dust_dataset_day_df = incheon_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)




In [ ]:
seoul_dataset_day_df

In [ ]:
seoul_dataset

In [102]:
seoul_dataset_day_df.to_csv('./data/seoul_pow_day2.csv')
pusan_dataset_day_df.to_csv('./data/pusan_pow_day2.csv')
jeju_dataset_day_df.to_csv('./data/jeju_pow3_day.csv')